<a href="https://colab.research.google.com/github/maerazor/Aircraft_Climb_Optimization/blob/master/Aircraft_Climb_V_Sep_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aircraft Climb Optimization 
(AIRBUS QUANTUM COMPUTING CHALLENGE)

In this document is the simplified statement problem published in September and turned to the International System of Units.

Given the ambiguity of some definitions, in this notebook assumes the following:



\begin{equation}
\rho(0) \equiv \rho (Zp_0) \not \equiv \rho_0
\end{equation}

\begin{equation}
\rho_i \equiv \rho (Zp_i)  \space\space\space _{(1 \leq i \leq N-2)}
\end{equation}

\begin{equation}
\rho_i \equiv \rho_0  \space\space\space _{(i = 0)}
\end{equation}

\begin{equation}
MACH(\nu_i,Zp_i) \equiv M(\nu,Zp)
\end{equation}

The steps followed were as follows
* I isolated $\lambda_{i+1}$ from the third equation and replaced it in the first one (red arrow).
* I isolated $Cz_{i+1}$ from the third equation and cleared it in the first (blue arrow).
* Given the values of $\nu_{i+1}$ and $\gamma_{i + 1}$ I found the value of $m_{i+1}$ such that first relation was satisfied, with this value and with the values $\nu_{i+1}$ and $\gamma_{i + 1}$ and also using the other relations I found $Cz_{i+1}$, $t_{i+1}$, $s_{i+1}$, $\lambda_{i+1}$

<div>
<img src="https://raw.githubusercontent.com/maerazor/Aircraft_Climb_Optimization/master/Untitled-1.jpg" width="700"/>
</div>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton
import scipy.optimize as optimize
from matplotlib import cm
import ipyvolume as ipv
# %matplotlib inline

In [0]:
# Table, pag: 6
Cx_o   =  0.014
k      =  0.09
Cz_max =  0.7
S_REF  =  120
eta    =  0.06/3600
Zp_I   =  3048
Zp_F   =  10972.8
m_I    =  60000
CAS_I  =  128.61111111
VMO    =  180.05555555
MMO    =  0.82
M_CRZ  =  0.80
L      =  400000
s_F    =  L
Vz_min =  1.524
g_o    =  9.80665
CI     =  30/60

# Defs, pag: 3
m_o     =  m_I
t_o     =  0
s_o     =  0
lamda_o =  1

# Defs (2) pag: 4
Ts_o   =  288.15
ρ_o    =  1.225
L_z    =  -0.0065
g_o    =  9.80665
R      =  287.05287 
a_o    =  -g_o/(R*L_z)

In [0]:
def Zp(i):
    return Zp_I + i*(Zp_F - Zp_I)/(N-1)

def F_N_MCL(i):
    return 140000.0 - 2.53*(Zp(i)/0.3048)

def ρ(i):
    return ρ_o * ((Ts_o + L_z*Zp(i))/Ts_o)**(a_o-1)

def M(l):
    return v[l]/np.sqrt(1.4*R*(Ts_o + L_z * Zp(l)))

def CAS(l):
    return np.sqrt(7*R*Ts_o*((((Ts_o + L_z*Zp(l))/Ts_o)**a_o * \
            ((1 + v[l]**2/(7*R*(Ts_o+L_z*Zp(l))))**3.5 - 1) + 1)**(1/3.5) - 1))

In [0]:
TAS_I = np.sqrt(7*R*(Ts_o + L_z*Zp_I)*((((Ts_o + L_z*Zp_I)/Ts_o)**-a_o * ((1 + CAS_I**2/(7*R*Ts_o))**3.5 - 1) + 1)**(1/3.5) - 1))
v_o   = TAS_I
N=53; Cz_o = m_o*g_o/(.5*ρ(0)*v_o**2*S_REF)
r_o  =  np.arcsin((F_N_MCL(0) - .5 *ρ(0)*v_o**2*S_REF*(Cx_o + k*m_o*g_o/(.5   *ρ(0)*v_o**2*S_REF))) / (m_o*g_o))

In [0]:
ρ_F = ρ_o * ((Ts_o + L_z*Zp_F)/Ts_o)**(a_o-1)
v_F = M_CRZ*np.sqrt(1.4*R*(Ts_o + L_z*Zp_F))

In [0]:
def Cz_ip (m_ip): #despejada de Ec2,  pag:3
    if i==0: ρo=ρ_o
    else:    ρo=ρ(i)
    return ((r[i+1]-r[i])*2/(Zp(i+1)-Zp(i)) + g_o/(v[i+1]**2*np.tan(r[i+1])) - .5*ρo*S_REF*Cz[i]/(m[i]*np.sin(r[i]))  \
            + g_o/(v[i]**2*np.tan(r[i])))   *   m_ip*np.sin(r[i+1])/(.5*ρ(i+1)*S_REF)

def g_v_i(m_ip):        
    return -v[i+1] + v[i]+(Zp(i+1)-Zp(i))/2 * (lamda_ip(m_ip)*F_N_MCL(i+1)/(m_ip*v[i+1]*np.sin(r[i+1])) \
                                        - .5* ρ(i+1) *v[i+1]*S_REF*(Cx_o + k*Cz_ip(m_ip)**2)/(m_ip*np.sin(r[i+1])) \
                                        - g_o/v[i+1] + 𝜆[i]*F_N_MCL(i)/(m[i]*v[i]*np.sin(r[i])) \
                                        - .5 * ρ(i)*v[i]*S_REF*(Cx_o + k*Cz[i]**2)/(m[i]*np.sin(r[i])) \
                                        - g_o/v[i]
                                          )
def g_s_i(s_ip):
    return -s_ip+s[i]+(Zp(i+1)-Zp(i))/2 * (1/np.tan(r[i+1]) + 1/np.tan(r[i]))


def g_t_i(t_ip):
    return -t_ip+t[i]+(Zp(i+1)-Zp(i))/2 * (1/(v[i+1]*np.sin(r[i+1])) + 1/(v[i]*np.sin(r[i])))


def lamda_ip(m_ip):  # lamda <= 1 para todo i ##Despejada de Ec3, pag:3  
    return ((m[i] - m_ip)*2/((Zp(i+1)-Zp(i))*eta) - 𝜆[i]*F_N_MCL(i)/(v[i]*np.sin(r[i]))) * v[i+1]*np.sin(r[i+1])/ F_N_MCL(i+1)

In [0]:
def Θ(N_,_𝑣,r_):
    global N, v, r, m, s, t, 𝜆, Cz, i, v_
    N=N_; P=True ; v=_𝑣; r=r_; v_=v[N-1]
    
    m = [m_o]; s=[s_o]; t=[t_o]; 𝜆=[lamda_o]; Cz=[Cz_o]
    Cz_i = Cz_o
    for i in range (0, N-1, 1):
        m.append(newton(g_v_i, m[i])) 
        Cz.append(Cz_ip (m[i+1]))
        𝜆.append(lamda_ip(m[i+1]))
        s.append(newton(g_s_i, s[i]))
        t.append(newton(g_t_i, t[i]))
        
        if 𝜆[i+1] > 1 or 𝜆[i+1] < 0 or Cz[i+1] > Cz_max or CAS(i+1) > VMO or M(i+1) > MMO or v[i+1]*np.sin(r[i+1]) < Vz_min:
            P=False; break            
    return 𝜆[N-1], m[N-1], Cz[N-1], s[N-1], t[N-1], P

In [0]:
def A(m_):
    return -ρ_F*S_REF*Cx_o/(2*m_) - 6*k*m_*g_o**2/(ρ_F*S_REF*v_**4)
    
def B(m_):
    return 16*k*m_*g_o**2/(ρ_F*S_REF*v_**3)

def C(m_):
    return F_N_MCL(N-1)/m_ - 12*k*m_*g_o**2/(ρ_F*S_REF*v_**2)

def D(m_):
    if B(m_)**2 - 4*A(m_)*C(m_) >= 0:
        return np.sqrt(B(m_)**2 - 4*A(m_)*C(m_))
    else:
        return np.nan

In [0]:
def t_B(t_):
    if np.absolute((2*A(m_)*v_ + B(m_))/D(m_)) < 1 and np.absolute((2*A(m_)*v_F + B(m_))/D(m_)) < 1:
        return t_ + (2/D(m_)) * (np.arctanh((2*A(m_)*v_ + B(m_))/D(m_)) - np.arctanh((2*A(m_)*v_F + B(m_))/D(m_)))
    else:
        return np.nan

def m_B(m_,lamda_):
    return m_ - eta*lamda_*F_N_MCL(N-1)*(t_B(t_)-t_)

def s_B(s_):
    loggint =  (D(m_) - 2*A(m_)*v_F - B(m_))/( D(m_)-2*A(m_)*v_-B(m_))
    if loggint >= 0.0:
        return s_ + 1/A(m_)*np.log( (D(m_) - 2*A(m_)*v_F - B(m_))/( D(m_)-2*A(m_)*v_-B(m_)) ) - (B(m_)+D(m_))/(2*A(m_))*(t_B(t_)-t_) 
    else:
        return np.nan

## Check:

In [0]:
def Φ():
    global v_, m_, Cz_, s_, t_, lamda_
    v_, m_, Cz_, s_, t_ , lamda_ = 223.61, 59042, 15, 168717.2, 880.8, 1
    t_F = t_B(t_) + (s_F-s_B(s_))/v_F
    m_F = m_B(m_,lamda_)*np.exp(-2*eta*g_o*np.sqrt(k*Cx_o)/v_F*(s_F-s_B(s_)))
    
    print("FNMCL_N-1 =",F_N_MCL(N-1),"\nρ_F = ", ρ_F,"\nA = ",A(m_),"\nB = ",B(m_),"\nC = ",C(m_), "\nt_B = ", t_B(t_), \
         "\nm_B = ", m_B(m_,lamda_), "\ns_B = ",s_B(s_), "\nm_F = ", m_F, "\ntF = ", t_F,"\n")
    
    return -m_F + CI*(t_B(t_) - s_B(s_)/v_F)

print ('\033[1m' +  '𝜙 = %s' %Φ())

FNMCL_N-1 = 48920.00000000003 
ρ_F =  0.36518323251251555 
A =  -3.318141604079352e-05 
B =  0.016687802999444994 
C =  -1.9701070034643147 
t_B =  992.8391410054124 
m_B =  58950.650753700254 
s_B =  194453.96402237617 
m_F =  58358.27165636876 
tF =  1863.2367192041615 

𝜙 = -58273.56583785164


The following function returns the $\phi$ value given the vectors $\nu = (\nu_0, \nu_1, \nu_2, . . ., \nu_{N-1} )$ and $\gamma = (\gamma_0, \gamma_1, \gamma_2, . . ., \gamma_{N-1} )$, for any value of N, i.e $\phi = \phi(\nu, \gamma)$.

With this, then the function really depends on 104 variables for N = 53, since $\nu_0$ and $\gamma_0$ are initially calculated, however given this approach there are no values $\nu_i$ and $\gamma_i$ for $i > 1$ such that $\phi$ is real, not even for $N = 3$

In [0]:
def Φ(V,r_):
    global v, r, lamda_, m_, Cz_, s_, t_, P ; P=True ; v=V; r=r_
    lamda_, m_, Cz_, s_, t_, P = Θ(N,v,r)
    m_F = m_B(m_,lamda_)*np.exp(-2*eta*g_o*np.sqrt(k*Cx_o)/v_F*(s_F-s_B(s_)))
    if P == True:
        return -m_F + CI*(t_B(t_) - s_B(s_)/v_F)
    else:
        return np.nan

In [0]:
N=2
Φ((v_o, 220), (r_o, 4*np.pi/180))

-58254.25417531683

In [0]:
N=3
v=(v_o,   230            ,   220         )
r=(r_o,   3.015*np.pi/180,   5*np.pi/180 )
Φ(v,r)

nan

In [0]:
N=2; ZZ={} 
for u in range(50, 300, 1):
    ZZ[u]=[]
    for w in range (1, 300, 1):
        rr=w*0.1*np.pi/180
        aa=Φ((v_o, u), (r_o, rr))
        ZZ[u].append(aa)
        
def ZZZ(X,Y):
    return ZZ[X][Y]

ZZZ = np.vectorize(ZZZ)
X = np.arange(50, 300, 1)
Y = np.arange(0, 299, 1)
X, Y = np.meshgrid(X, Y)
Z = np.array([ZZZ(X,Y)])[0]
Y = (Y + 1)*0.1

The following is the graph of the $\phi$ function for $N=2$, where $\nu_1 [m/s]$ in $X$ and $\gamma_1 [Grad]$ in $Y$

In [0]:
ipv.figure()
mesh = ipv.plot_surface(X, Y, Z )
ipv.xlim(180, 250)
ipv.ylim(0, 8)
ipv.zlim(-59000, -57000)
ipv.show()

<div>
<img src="https://raw.githubusercontent.com/maerazor/Aircraft_Climb_Optimization/master/ipyvolume.png" width="700"/>
</div>

In [0]:
N=2
def f(params):
    a, b  = params
    #global v, r
    v=(v_o, a)
    r=(r_o, b)
    return -Φ(v,r) 

initial_guess = [190, 1*np.pi/180]
result = optimize.minimize(f, initial_guess, method = 'Nelder-Mead')

if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


[1.8832748e+02 1.7284241e-02]


Minimum value of $\phi$ for N = 2 is where the values of $\nu_1$ and $\gamma_1$ are $1.8832748e02$ and $1.7284241e-02$ respectively.

In [0]:
N=2
Φ((v_o, 188.32748), (r_o, 0.017284241))  #valor minimo de phi para N=2

-56762.46625254673

Thus, just by determining the vectors v and r, all other variables are completely determined.

In [0]:
print("\033[1mFor N = 2 \033[0m")
print("𝜈 = ",v)
print("𝛾 = ",r)
print("Cz = ",Cz)
print("m = ",m)
print("𝜆 = ",𝜆)
print("s = ",s)
print("t = ",t)

For N = 2 
𝜈 =  (148.52130232621022, 188.32748)
𝛾 =  (0.07652259401812767, 0.017284241)
Cz =  [0.4914381674079498, 0.6409684098068081]
m =  [60000, 58441.948437762374]
𝜆 =  [1, 0.8976028306703161]
s =  [0, 280906.13645509206]
t =  [0, 1566.3339526820662]
